## Xarray engine: extra dimensions

In [1]:
import earthkit.data as ekd

### 2D wave spectra example

We analyse a 2D wave spectra product at 2025-12-15 00 UTC and 03 UTC.
A specific feature of this dataset is that the fields are additionally
indexed by **wavelength** and **frequency**, on top of the standard
temporal dimension.

In [2]:
ds_fl = ekd.from_source("sample", "2d-wave-spectra_an.grib")

In [3]:
ds_fl.ls(extra_keys=["directionNumber", "frequencyNumber"])

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType,directionNumber,frequencyNumber
0,ecmf,2dfd,meanSea,0,20251215,0,0,an,0,regular_ll,1,1
1,ecmf,2dfd,meanSea,0,20251215,0,0,an,0,regular_ll,2,1
2,ecmf,2dfd,meanSea,0,20251215,0,0,an,0,regular_ll,3,1
3,ecmf,2dfd,meanSea,0,20251215,0,0,an,0,regular_ll,4,1
4,ecmf,2dfd,meanSea,0,20251215,0,0,an,0,regular_ll,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2083,ecmf,2dfd,meanSea,0,20251215,300,0,an,0,regular_ll,32,29
2084,ecmf,2dfd,meanSea,0,20251215,300,0,an,0,regular_ll,33,29
2085,ecmf,2dfd,meanSea,0,20251215,300,0,an,0,regular_ll,34,29
2086,ecmf,2dfd,meanSea,0,20251215,300,0,an,0,regular_ll,35,29


To represent this structure in Xarray, the predefined dimensions of the
Xarray engine must therefore be complemented with dimensions derived
from the metadata keys ``"directionNumber"`` and ``"frequencyNumber"``.

In [4]:
ds = ds_fl.to_xarray(
    profile="grib", 
    extra_dims=["directionNumber", "frequencyNumber"], 
    add_earthkit_attrs=False, 
)
ds

<xarray.Dataset> Size: 1MB
Dimensions:                  (directionNumber: 36, frequencyNumber: 29,
                              forecast_reference_time: 2, latitude: 7,
                              longitude: 12)
Coordinates:
  * directionNumber          (directionNumber) int64 288B 1 2 3 4 ... 34 35 36
  * frequencyNumber          (frequencyNumber) int64 232B 1 2 3 4 ... 27 28 29
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 16B 202...
  * latitude                 (latitude) float64 56B 90.0 60.0 ... -60.0 -90.0
  * longitude                (longitude) float64 96B 0.0 30.0 ... 300.0 330.0
Data variables:
    2dfd                     (directionNumber, frequencyNumber, forecast_reference_time, latitude, longitude) float64 1MB ...
Attributes:
    Conventions:  CF-1.8
    institution:  ECMWF

The ``extra_dims`` option also supports defining an explicit mapping
between the name of an extra dimension and the corresponding metadata
key, in a way that is conceptually similar to **dimension roles**.

In [5]:
ds2 = ds_fl.to_xarray(
    profile="grib", 
    extra_dims=[
        {"d": "directionNumber"}, 
        {"f": "frequencyNumber"}
    ],
    add_earthkit_attrs=False, 
)
ds2

<xarray.Dataset> Size: 1MB
Dimensions:                  (d: 36, f: 29, forecast_reference_time: 2,
                              latitude: 7, longitude: 12)
Coordinates:
  * d                        (d) int64 288B 1 2 3 4 5 6 7 ... 31 32 33 34 35 36
  * f                        (f) int64 232B 1 2 3 4 5 6 7 ... 24 25 26 27 28 29
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 16B 202...
  * latitude                 (latitude) float64 56B 90.0 60.0 ... -60.0 -90.0
  * longitude                (longitude) float64 96B 0.0 30.0 ... 300.0 330.0
Data variables:
    2dfd                     (d, f, forecast_reference_time, latitude, longitude) float64 1MB ...
Attributes:
    Conventions:  CF-1.8
    institution:  ECMWF

### Quantiles in a probabilistic forecast

Let us now consider a probabilistic forecast of 2-metre temperature.

In [6]:
ds_fl2 = ekd.from_source("sample", "quantiles_pd.grib")

In this dataset, the fields are indexed by the metadata key ``"quantile"``.

In [7]:
ds_fl2.ls(keys=[
    "shortName", 
    "dataDate", 
    "dataTime", 
    "stepRange", 
    "dataType", 
    "quantile", 
    "number", 
    "numberOfForecastsInEnsemble"
])

,shortName,dataDate,dataTime,stepRange,dataType,quantile,number,numberOfForecastsInEnsemble
0,2tp,20251209,0,0-168,pd,1:3,1,3
1,2tp,20251209,0,0-168,pd,1:5,1,5
2,2tp,20251209,0,0-168,pd,1:10,1,10
3,2tp,20251209,0,0-168,pd,2:3,2,3
4,2tp,20251209,0,0-168,pd,2:5,2,5
5,2tp,20251209,0,0-168,pd,2:10,2,10
6,2tp,20251209,0,0-168,pd,3:3,3,3
7,2tp,20251209,0,0-168,pd,3:5,3,5
8,2tp,20251209,0,0-168,pd,3:10,3,10
9,2tp,20251209,0,0-168,pd,4:5,4,5


Note that, in this context, the usual meaning of the metadata key ``"number"`` (and the related ``"numberOfForecastsInEnsemble"``) is overridden by ``"quantile"``. As a result, the ensemble dimension normally derived from ``"number"`` is no longer applicable.

For this reason, we must:

- declare ``"quantile"`` as an extra dimension, and

- remove the predefined ensemble dimension ``"number"``, since it would otherwise conflict with the ``"quantile"`` dimension.

In [8]:
ds3 = ds_fl2.to_xarray(
    profile="grib", 
    squeeze=False, 
    extra_dims="quantile", 
    drop_dims="number", 
    add_earthkit_attrs=False, 
)
ds3

<xarray.Dataset> Size: 13kB
Dimensions:                  (quantile: 18, forecast_reference_time: 1,
                              step: 1, level: 1, level_type: 1, latitude: 7,
                              longitude: 12)
Coordinates:
  * quantile                 (quantile) <U5 360B '10:10' '1:10' ... '9:10'
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 8B 2025...
  * step                     (step) timedelta64[ns] 8B 7 days
  * level                    (level) int64 8B 0
  * level_type               (level_type) <U7 28B 'surface'
  * latitude                 (latitude) float64 56B 90.0 60.0 ... -60.0 -90.0
  * longitude                (longitude) float64 96B 0.0 30.0 ... 300.0 330.0
Data variables:
    2tp                      (quantile, forecast_reference_time, step, level, level_type, latitude, longitude) float64 12kB ...
Attributes:
    Conventions:  CF-1.8
    institution:  ECMWF

### The option ``ensure_dims`` vs ``extra_dims``

The ``extra_dims`` and ``ensure_dims`` options partially overlap in their usage - when introducing a new dimension that must not be squeezed, it is sufficient to list it in ``ensure_dims``. In this case, there is no need to repeat the same dimension in ``extra_dims``.

In [9]:
ds4 = ds_fl2.sel(quantile="2:3").to_xarray(
    profile="grib", 
    squeeze=True, 
    ensure_dims="quantile", 
    drop_dims="number", 
    add_earthkit_attrs=False, 
)
ds4

<xarray.Dataset> Size: 836B
Dimensions:    (quantile: 1, latitude: 7, longitude: 12)
Coordinates:
  * quantile   (quantile) <U3 12B '2:3'
  * latitude   (latitude) float64 56B 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * longitude  (longitude) float64 96B 0.0 30.0 60.0 90.0 ... 270.0 300.0 330.0
Data variables:
    2tp        (quantile, latitude, longitude) float64 672B ...
Attributes:
    Conventions:  CF-1.8
    institution:  ECMWF